In [1]:
from pathlib import Path
import sys
import json


# Поднимаемся на два уровня вверх (из notebooks/ в InteriorClass/)
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))  # Теперь Python увидит src/


import torch
from src.config import RANDOM_SEED, SPLIT_CONFIG, MIN_VAL_TEST_PER_CLASS, CLASS_LABELS
from src.schemas.configs import ExperimentConfig
from src.experiment import Experiment


In [2]:
# 1. =========================== Define hyperparameters ===========================
EXP_NUMBER = 16
BATCH_SIZE = 32
EPOCHS = 10
START_LR = 3e-5
IMG_SIZE = 448
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# 2. ================================ Define paths =================================
data_dir = project_root / "data"
print(f"data_dir: {data_dir}")
dataset_dir = data_dir / "interior_dataset"

exp_dir = project_root / "experiments" / f"exp{EXP_NUMBER:03d}"

exp = Experiment(
    dataset_dir=dataset_dir,
    exp_dir=exp_dir,
    class_labels=CLASS_LABELS,
    splits=SPLIT_CONFIG,
    exp_number=EXP_NUMBER,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    img_size=IMG_SIZE,
    start_lr=START_LR,
    random_seed=RANDOM_SEED,
    device=DEVICE
)

data_dir: /home/little-garden/CodeProjects/InteriorClass/data


Total samples: 45724
Train samples: 41745
Val samples: 1983
Test samples: 1996


In [4]:
import shutil
from pathlib import Path
from typing import List
from src.schemas.types import SampleItem

def copy_test_dataset(test_samples: List[SampleItem], output_dir: Path) -> None:
    """
    Копирует все изображения из test_samples в output_dir/test_dataset/
    
    Args:
        test_samples: список SampleItem из test_dataset
        output_dir: корневая директория для копирования (например, Path('/home/little-garden/CodeProjects/InteriorClass/data/interior_dataset/'))
    """
    # Создаем папку test_dataset
    test_dataset_dir = output_dir / "test_dataset"
    test_dataset_dir.mkdir(exist_ok=True)
    
    print(f"Копирую {len(test_samples)} файлов в {test_dataset_dir}")
    
    for i, sample in enumerate(test_samples):
        # Получаем относительный путь от корневой директории
        relative_path = sample.filepath.relative_to(output_dir)
        
        # Создаем путь назначения
        dest_path = test_dataset_dir / relative_path
        
        # Создаем папки если нужно
        dest_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Копируем файл
        shutil.copy2(sample.filepath, dest_path)
        
        if (i + 1) % 100 == 0:
            print(f"Скопировано {i + 1}/{len(test_samples)} файлов")
    
    print(f"Готово! Все файлы скопированы в {test_dataset_dir}")


In [5]:
# Получи test_samples из твоего эксперимента
test_samples = exp.test_dataset.sample_items

# Скопируй файлы
copy_test_dataset(
    test_samples=test_samples,
    output_dir=Path('/home/little-garden/CodeProjects/InteriorClass/data/interior_dataset/')
)

Копирую 1996 файлов в /home/little-garden/CodeProjects/InteriorClass/data/interior_dataset/test_dataset
Скопировано 100/1996 файлов
Скопировано 200/1996 файлов
Скопировано 300/1996 файлов
Скопировано 400/1996 файлов
Скопировано 500/1996 файлов
Скопировано 600/1996 файлов
Скопировано 700/1996 файлов
Скопировано 800/1996 файлов
Скопировано 900/1996 файлов
Скопировано 1000/1996 файлов
Скопировано 1100/1996 файлов
Скопировано 1200/1996 файлов
Скопировано 1300/1996 файлов
Скопировано 1400/1996 файлов
Скопировано 1500/1996 файлов
Скопировано 1600/1996 файлов
Скопировано 1700/1996 файлов
Скопировано 1800/1996 файлов
Скопировано 1900/1996 файлов
Готово! Все файлы скопированы в /home/little-garden/CodeProjects/InteriorClass/data/interior_dataset/test_dataset
